In [109]:
import pandas as pd
from pandas.io import gbq
import numpy as np
import os
import glob
import re

In [110]:
import pydata_google_auth

In [111]:
    connect_bq = [
        'https://www.googleapis.com/auth/cloud-platform',
        'https://www.googleapis.com/auth/drive',
    ]

    credentials = pydata_google_auth.get_user_credentials(
        connect_bq,
        auth_local_webserver=True,
    )

#### Query that retrieves add_ons per user for a month and the add ons that a bundle has

In [112]:
sql_query = """
CREATE TEMP FUNCTION STRING_DEDUP(str STRING) AS (
(SELECT STRING_AGG(item ORDER BY item) FROM (
SELECT DISTINCT item FROM UNNEST(SPLIT(str)) item 
)) 
);
----------------------------------------------------------------------------------------------------------------------
with daily_status as (
Select * , DATE_TRUNC(day, month) as change_month
from `fubotv-prod.data_insights.daily_status_static_update` t1
where final_status_restated like '%paid%'
and day >= '2021-01-01'
AND day <= current_date()-1
),
add_on as (
select account_code, change_month, STRING_DEDUP(add_ons) as addons_sorted,STRING_DEDUP(bundle_add_on_list) as bundle_addons_sorted
FROM daily_status
)
select DISTINCT change_month as month, account_code, bundle_addons_sorted, addons_sorted
from add_on
"""

In [113]:
df = pd.read_gbq(
    sql_query,
    project_id='fubotv-prod',
    credentials=credentials,
)

#### Checking / Housekeeping

In [114]:
df.head()

,month,account_code,bundle_addons_sorted,addons_sorted
0,2022-06-01,62138c861d2a050001cf8773,"advanced-dvr-1000,third-screen,unlimited-scree...","advanced-dvr-1000,amc-premiere,starz-epix-show..."
1,2022-04-01,5b7b3dc5e09d61ef1b4a74ed,"advanced-dvr,third-screen","advanced-dvr,deportes,rsn-fee,third-screen"
2,2021-06-01,5fe4ee8ff0680c0001514ab5,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,epix,fubo-extra-lite,third-s..."
3,2022-05-01,61f335130ffeb20001e191c1,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,news-plus,th..."
4,2022-03-01,6133a334439c4e000195c01b,"advanced-dvr-250,third-screen","advanced-dvr-250,fubo-extra-lite,rsn-fee,third..."


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18296490 entries, 0 to 18296489
Data columns (total 4 columns):
 #   Column                Dtype         
---  ------                -----         
 0   month                 datetime64[ns]
 1   account_code          object        
 2   bundle_addons_sorted  object        
 3   addons_sorted         object        
dtypes: datetime64[ns](1), object(3)
memory usage: 558.4+ MB


##### if the bundles addon column is empty, discard that row

In [116]:
df = df.dropna()

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17269293 entries, 0 to 18296489
Data columns (total 4 columns):
 #   Column                Dtype         
---  ------                -----         
 0   month                 datetime64[ns]
 1   account_code          object        
 2   bundle_addons_sorted  object        
 3   addons_sorted         object        
dtypes: datetime64[ns](1), object(3)
memory usage: 658.8+ MB


##### Comparing bundle add_ons to the add_ons field and identifying the difference between these two fields and joining each add_on by a comma to a new field called new_addition

In [118]:
df['nonbundle_addons'] = [','.join(set(n.split(',')) - set(o.split(','))) 
                                                          for o, n in zip(df.bundle_addons_sorted, df.addons_sorted)]

In [119]:
df

,month,account_code,bundle_addons_sorted,addons_sorted,nonbundle_addons
0,2022-06-01,62138c861d2a050001cf8773,"advanced-dvr-1000,third-screen,unlimited-scree...","advanced-dvr-1000,amc-premiere,starz-epix-show...","starz-epix-showtime,amc-premiere"
1,2022-04-01,5b7b3dc5e09d61ef1b4a74ed,"advanced-dvr,third-screen","advanced-dvr,deportes,rsn-fee,third-screen","deportes,rsn-fee"
2,2021-06-01,5fe4ee8ff0680c0001514ab5,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,epix,fubo-extra-lite,third-s...",epix
3,2022-05-01,61f335130ffeb20001e191c1,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,news-plus,th...",news-plus
4,2022-03-01,6133a334439c4e000195c01b,"advanced-dvr-250,third-screen","advanced-dvr-250,fubo-extra-lite,rsn-fee,third...","rsn-fee,fubo-extra-lite"
...,...,...,...,...,...
18296485,2022-01-01,6175bfedb4f95f0001a132f4,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
18296486,2021-09-01,60610971c641910001edca22,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
18296487,2022-02-01,60c0cb33bff11a0001df9945,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
18296488,2022-01-01,6000b77f01dad2000188a4d3,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee


#### Splitting new_addition column into additional columns and exploding them into individual rows

In [120]:
df['nonbundle_addons'] = df['nonbundle_addons'].str.split(',')
df = df.explode('nonbundle_addons').reset_index(drop=True)
cols = list(df.columns)
df = df[cols]

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18767739 entries, 0 to 18767738
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   month                 datetime64[ns]
 1   account_code          object        
 2   bundle_addons_sorted  object        
 3   addons_sorted         object        
 4   nonbundle_addons      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 715.9+ MB


In [122]:
df.head()

,month,account_code,bundle_addons_sorted,addons_sorted,nonbundle_addons
0,2022-06-01,62138c861d2a050001cf8773,"advanced-dvr-1000,third-screen,unlimited-scree...","advanced-dvr-1000,amc-premiere,starz-epix-show...",starz-epix-showtime
1,2022-06-01,62138c861d2a050001cf8773,"advanced-dvr-1000,third-screen,unlimited-scree...","advanced-dvr-1000,amc-premiere,starz-epix-show...",amc-premiere
2,2022-04-01,5b7b3dc5e09d61ef1b4a74ed,"advanced-dvr,third-screen","advanced-dvr,deportes,rsn-fee,third-screen",deportes
3,2022-04-01,5b7b3dc5e09d61ef1b4a74ed,"advanced-dvr,third-screen","advanced-dvr,deportes,rsn-fee,third-screen",rsn-fee
4,2021-06-01,5fe4ee8ff0680c0001514ab5,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,epix,fubo-extra-lite,third-s...",epix


In [123]:
df = df.drop(['addons_sorted', 'bundle_addons_sorted'], axis=1)

In [124]:
df = df.drop_duplicates()

In [125]:
df

,month,account_code,nonbundle_addons
0,2022-06-01,62138c861d2a050001cf8773,starz-epix-showtime
1,2022-06-01,62138c861d2a050001cf8773,amc-premiere
2,2022-04-01,5b7b3dc5e09d61ef1b4a74ed,deportes
3,2022-04-01,5b7b3dc5e09d61ef1b4a74ed,rsn-fee
4,2021-06-01,5fe4ee8ff0680c0001514ab5,epix
...,...,...,...
18767734,2022-01-01,6175bfedb4f95f0001a132f4,rsn-fee
18767735,2021-09-01,60610971c641910001edca22,rsn-fee
18767736,2022-02-01,60c0cb33bff11a0001df9945,rsn-fee
18767737,2022-01-01,6000b77f01dad2000188a4d3,rsn-fee


#### Aggregating by month and new_addition to count distinct account_code

In [126]:
df = df.groupby(['month', 'nonbundle_addons'],  as_index=False ).agg({'account_code': 'nunique'})

In [127]:
df = df[df.nonbundle_addons != 'rsn-fee']
df = df[df.nonbundle_addons != '']
df = df[df.nonbundle_addons != ""]

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522 entries, 1 to 557
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   month             522 non-null    datetime64[ns]
 1   nonbundle_addons  522 non-null    object        
 2   account_code      522 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 16.3+ KB


In [107]:
df.to_csv('/Users/nanditanandakumar/Dropbox (fuboTV)/Business Analytics/Team/Nandita/Git/Addons_Project/NonBundle_AddOns - Count.csv', index =False)

In [129]:
df.to_gbq ( destination_table='business_analytics.temp_nonbundle_addon_counts', project_id= 'fubotv-dev', if_exists= 'replace')